In [1]:
import time

import numpy as np

from file_handler import readTxtFile, readXlsFile
from quantify_variability import excludeGenes, mainStatisticsSummaryFile, mergeGenesWithChannels, calculateVariabilityScores, unifyMetricsNormalizeAndProduceSingleScore
#from quantify_variability import filterLowAndHighMean

# Cell by gene file or files folder location
NORMALIZED_DATA_PATH = r"Z:\danielda\analysis\zp_p2f_auto_240924\analysis\variability-quantifier\p2f"
# Genes to readouts and channels folder
CHANNELS_FILE_PATH = r"Z:\danielda\analysis\zp_p2f_auto_240924\analysis\variability-quantifier\p2f\gene_lists"
# This is the first gene column in the cell by gene file (6 for par-seq, 7 for par²-seq because of additional "library" column)
GENE_FIRST_COL = 7
# Split per dilution of the culture
SPLIT_PER_DILUTION = False
# par-seq: False and empty list of control genes. par²-seq: True or False, filled list
CONTROL_PER_LIBRARY = False
# All genes that has a "gene_class" in "genes_to_readouts" files. Use if you want them to be for each library separately
CONTROL_GENES_ETC = ['ackA2probes', 'cre', 'pro', 'dbn', 'dnaA_A647', 'dnaA_A550', 'dnaA_A488', 'tRNA', 'tmRNA', '6S_ncRNA', '23S_rRNA', '5S_ncRNA', 'pre_16s', 'fimA', 'slp', 'rpsC', 'rpoS', 'recA', 'ackA', 'gadB', 'cheA', 'fliC']
# If par and not par^2, use empty list
#CONTROL_GENES_ETC = []
# Assign True if there is already "statistics_summary.csv" file in your directory
LOAD_FILE = True
# List of genes to exclude from analysis. this includes control genes etc.
#GENES_TO_EXCLUDE = ['cre', 'pro', 'dbn', 'exaB', 'katB', 'pvdQ', 'pqsC', 'algD', 'pscQ', 'PA3281', 'pscF', 'PA1697', 'thyA', 'PA1697', 'PA0485', 'pelA', 'pgi', 'lecA', 'chiC', 'ambB', 'rbsK', 'ampR', 'xcpP', 'exsC']
GENES_TO_EXCLUDE = []
# Mean expression cutoff where genes below min and above max are filtered
MIN_MEAN_EXPRESSION_THRESH_A488 = 2**-8
MIN_MEAN_EXPRESSION_THRESH_A550 = 2**-8
MIN_MEAN_EXPRESSION_THRESH_A647 = 2**-8
MAX_MEAN_EXPRESSION_THRESH_A488 = 2**15
MAX_MEAN_EXPRESSION_THRESH_A550 = 2**15
MAX_MEAN_EXPRESSION_THRESH_A647 = 2**15
CONSTRAINTS_PER_CHANNEL = {'A488': (MIN_MEAN_EXPRESSION_THRESH_A488, MAX_MEAN_EXPRESSION_THRESH_A488),
                           'A550': (MIN_MEAN_EXPRESSION_THRESH_A550, MAX_MEAN_EXPRESSION_THRESH_A550),
                           'A647': (MIN_MEAN_EXPRESSION_THRESH_A647, MAX_MEAN_EXPRESSION_THRESH_A647)}
# If cell is expressing gene higher than this constant, this measurement is dropped
SD_ABOVE_MEAN_THRESH = 100000
# Genes that expressed by lower number of cells than this constant are filtered
MINIMUM_CELLS_EXPRESSING = 0
# Drop genes that are above this threshold
MINIMUM_PROBES_PER_GENE = 2
# x-axis in the residual analysis
RESIDUALS_X_AXIS = 'log₂(mean expression)'
# Calculate residuals from polynomial where y-axis is:
RESIDUALS_Y_AXIS = ['log₂(CV)',
                    'log₂(top/bottom ratio)',
                    'log₂(kurtosis)',
                    'log₂(entropy)',
                    'percent expressing'
                    ]
# Size of the subpopulations in tails
TOP_TO_BOTTOM_RATIO_TAIL = 0.05
# Positive residual implies variability in: 'log₂(CV)', 'log₂(top/bottom ratio)', 'log₂(kurtosis)'
# Negative residual implies variability in: 'log₂(entropy)', 'percent expressing'
# Calculate the residuals from different fit for each channel
FIT_PER_CHANNEL = True
# Calculate residuals from the closest confidence interval
ROBUST_RESIDUALS = True
# Confidence interval for the polynomial fits. Equals to (1 - α)
ALPHA = 0.05
# For each metric - its own polynomial complexity
POLYNOMIAL_DEGREES = {'log₂(CV)':              2,
                      'log₂(top/bottom ratio)':3,
                      'log₂(kurtosis)':        2,
                      'log₂(entropy)':         2,
                      'percent expressing':    -1}

In [2]:
# Load cell by gene file, normalized by probe number
norm_data = readTxtFile(NORMALIZED_DATA_PATH, CONTROL_GENES_ETC, CONTROL_PER_LIBRARY)

Loading cell by gene runtime: 4.379033803939819 seconds


In [3]:
# Load channels table, with columns: "channel" and "gene_name"
channels_table = readXlsFile(CHANNELS_FILE_PATH, CONTROL_GENES_ETC, CONTROL_PER_LIBRARY)


Loading genes list runtime: 1.094411849975586 seconds


In [4]:
excludeGenes(norm_data, GENES_TO_EXCLUDE)

In [5]:
# If statistics file exist, load it. if not, create it
if LOAD_FILE:
    summary_table = mainStatisticsSummaryFile(LOAD_FILE)
else:
    norm_data_channel = mergeGenesWithChannels(channels_table, norm_data, CONTROL_GENES_ETC, GENE_FIRST_COL,
                                                   CONTROL_PER_LIBRARY, SPLIT_PER_DILUTION)
    # If we want to create new file
    summary_table = mainStatisticsSummaryFile(LOAD_FILE, norm_data_channel, GENE_FIRST_COL, SD_ABOVE_MEAN_THRESH,
                                                  MINIMUM_CELLS_EXPRESSING, CONTROL_GENES_ETC,
                                                  CONTROL_PER_LIBRARY, TOP_TO_BOTTOM_RATIO_TAIL, CONSTRAINTS_PER_CHANNEL, MINIMUM_PROBES_PER_GENE)

Loading main statistics summary...
Statistics summary loaded.


In [6]:
start_time = time.time()
#summary_table = filterLowAndHighMean(summary_table, CONSTRAINTS_PER_CHANNEL)
res_table = calculateVariabilityScores(summary_table, RESIDUALS_X_AXIS,                                                      RESIDUALS_Y_AXIS, FIT_PER_CHANNEL,
                                       ROBUST_RESIDUALS, ALPHA, POLYNOMIAL_DEGREES)
variability_score = unifyMetricsNormalizeAndProduceSingleScore(res_table, RESIDUALS_Y_AXIS)
end_time = time.time()
print('Calculating variability scores runtime:', end_time - start_time)


Polynomial of degree 2 coefficients with log₂(CV) as y axis:
Intercept         2.577896
np.power(x, 1)   -0.452918
np.power(x, 2)    0.004775
dtype: float64

Polynomial of degree 2 coefficients with log₂(CV) as y axis:
Intercept         2.306757
np.power(x, 1)   -0.432081
np.power(x, 2)    0.003026
dtype: float64

Polynomial of degree 2 coefficients with log₂(CV) as y axis:
Intercept         2.413450
np.power(x, 1)   -0.433796
np.power(x, 2)    0.002389
dtype: float64

Polynomial of degree 3 coefficients with log₂(top/bottom ratio) as y axis:
Intercept         1.851462
np.power(x, 1)    0.225614
np.power(x, 2)   -0.004415
np.power(x, 3)    0.000246
dtype: float64

Polynomial of degree 3 coefficients with log₂(top/bottom ratio) as y axis:
Intercept         2.567746
np.power(x, 1)    0.229095
np.power(x, 2)   -0.009696
np.power(x, 3)    0.000913
dtype: float64

Polynomial of degree 3 coefficients with log₂(top/bottom ratio) as y axis:
Intercept         2.479219
np.power(x, 1)    0.23700